# SFT数据构造
## 数据蒸馏
对ERNIE-Speed做微调，我们可以用ERNIE-3.5、4.0回答问题。
## Self-QA
有100个文档，从文档中找出知识点，可以通过大模型的方式，从文档中进行题目抽取。

In [14]:
# 1、确定我们要抽取的数据类型。问答对、选择题、填空题、判断题
# 2、调试prompt
prompt = f"""
你是一个电力行业的专家，你的任务是从一段电力行业文本资料中抽取一些问答题，具体要求如下：
1、问题答案需要准确无误，问题、答案要求尽量来源于原文内容，不要改写
2、不要提出对于文档、原文的总结性问题，而是聚焦于具体的知识点
3、输出的每个问题需要包含：问题、分析、答案、依据
4、输出格式为json格式，[{{"问题":"","分析":"","答案":"","依据":""}}]

##示例
输入：水能是一种能源，是清洁能源.是绿色能源，是指水体的动能、势能和压力能等能量资源。水能是一种可再生能源，水能主要用于水力发电。水力发电将水的势能和动能装换成电能。以水力发电的工厂称为水力发电厂，简称水电厂，又称水电站，水力发电的优点是成本低、可连续再生、无污染。缺点是分布受水文、气候、地貌等自然条件的限制大。容易被地形、气候等多方面的因素所影响，国家还在研究如何更好的利用水能。广义的水能资源电活可流水域、蓄夕水能、放旋越、海越能等能是意义的水能资可流的水能资,是能源一难源,我们的才能是钢动能人们是易开放用的域成重的能也是确流能源。
水落差在重力作用下形能,从河流或水车转等事立水源河立水利脉水的压力或者速材水物机之,从而常水增转状水机能燃后再水物将带动发电机旋转能磁力线产生交流转。而低处的水通过阳光照射，形成水蒸气，循环到地球各处从而恢复高位水源的水分布。水不仅可以直接被人类利用，它还是能量的载体，太阳能驱动地球上水循环使之持续进行，地表水的流动是重要的一环在落差大、流量大的地区，水能资源丰富。随着矿物燃料的日渐减少，水能是非常重要目前景广阔的替代资源，世界上水力发电还处于起步阶段。河流、潮汐、波浪以及涌浪等水运动均可以用来发电，也有部分水能用于灌溉。
输出：
```json
[{{"问题":"水能的主要应用场景是什么？","分析":"原文中提到“水能是一种可再生能源水能主要用于水力发电”，因此水能主要应用于水力发电。","答案":"水能主要用于水力发电","依据":"水能是一种可再生能源，水能主要用于水力发电。"}},{{"问题":"水力发电的优点是什么？","分析":"原文中提到“水力发电的优点是成本低、可连续再生、无污染。”，所以得出水力发电的优点是成本低、可连续再生、无污染。","答案":"水力发电的优点是成本低、可连续再生、无污染","依据":"水力发电的优点是成本低、可连续再生、无污染。"}}]
```

输入：空气是风冷散热系统最常用的冷却介质此外，还可以采用气和氮气对电机进行冷却，XU等采用氟气对高温气冷堆的鼓风机驱动电机进行冷却，电机额定功率为4500kW,仿真结果表明氮气的初始温度显著影响电机的温升，氮气和氢气的热导率是空气的6-7倍，因此相同的气体温度和流速情况下氮气、氩气冷却的散热效率可以达到控气的1.3-1.5倍，同时氢气和氟气的密度只有空气的1/14-177因此采用氢气、氮气代替空气可以显著减小冷却系统的通风损耗和风摩损耗，然而氢气和氮气高昂的价格、复杂的储存供应系统限制了氢气、氮气冷却系统的大规模应用，此外氢气冷却系统需要氢气纯度不小于96%和安全可靠的密封防爆系统，这进一步限制了氢气冷却系统的应用。因此，氢气、氨气冷却系统多应用于大功率发电机、高温反应堆等特殊场合。
输出：
```json
[{{"问题":"电机散热系统中，采用氢气或氟气作为冷却介质的优势和限制分别是什么？","分析":"在原文中提到“氮气和氯气的热导率是空气的6-7倍，因此相同的气体温度和流速情况下氮气、氯气冷却的散热效率可以达到空气的1.3-1.5倍。同时氢气和氮气的密度只有空气的1/14-1/7因此采用氢气、氮气代替空气可以显著减小冷却系统的通风损耗和风摩损耗。”、“然而氢气和气高昂的价格、复杂的储存供应系统限制了氩气、氩气冷却系统的大规模应用。此外氢气冷却系统需要氢气纯度不小于96%和安全可靠的密封防爆系统这进一步眼制了氢气冷却系统的应用”等描述，因此得出答案是“使用氢气或气作为冷却介质的优势是：氮气和氢气的热导率是空气的6-7倍因此相同的气体温度和流速情况下氟气、氮气冷却的散热效率可以达到空气的1.3-15倍。同时，氢气和氮气的密度只有空气的1/14-1/7因此采用氢气、氢气代替空气可以显著减小冷却系统的通风损耗和风摩损耗；限制是：氢气和氮气高昂的价格、复杂的储存供应系统限制了氢气、氮气冷系统的大规模应用、此外氢气冷却系统需要氢气纯度不小于96%和安全可靠的密封防爆系统这进一步限制了氮气冷却系统的应用","答案":"使用氢气或氟气作为冷却介质的优势是：气和氢气的热导率是空气的6-7倍因此相同的气体温度和流速情况下，氮气、氮气冷却的散热效率可以达到空气的1.3-1.5倍。同时氢>气和氮气的密度只有空气的1/14-1/7因此采用氮气、氢气代替空气可以显著减小冷却系统的通风损耗和风摩损耗；限制是：氢气和氮气高昂的价格、复杂的储存供应系统限制了氮气、气冷却系统的大规模应用。此外，氢气冷却系统需要氢气纯度不小于96%和安全可靠的密封防爆系统这进一步限制了氢气冷却系统的应用","依据":"气和氢气的热导率是空气的6-7倍，因此相同的气体温度和流速情况下，氟气、氩气冷却的散热效率可以达到空气的1.3-1.5倍，同时氢气和氟气的密度只有空气的1/14-1/7因此采用氢气、氮气代替空气可以显著减小冷却系统的通风损耗和风摩损耗，然而氩气和氮气高昂的价格、复杂的储存供应系统限制了氩气、氟气冷却系统的大规模应用，此外氢气冷却系统需要氢气纯度不小于96%和安全可靠的密封防爆系统，这进一步限制了氢气冷却系统的应用"}}]
```

输入：客户服务中心：
当前，公司面临电力改革、分布式电源迅速发展的新形势，
市场开拓和优质服务压力越来越大，迫切需要进一步解放思想，转变观念，创新机制，以客户需求为导向，精简办电程序，确保公司市场竞争优势。2014年，各单位认真落实《国家电网公司关于简化业扩手续提高办电效率深化为民服务的工作意见》（国家电网营销〔2014〕1049号）,提升了业扩报装效率、工作质量和服务水平。为深入贯彻“你用电、我用心”服务理念，最大限度地实现供电服务便民、为民、利民，提升优质服务水平，加快办电速度，公司研究制订了《进一步精简业扩手续、提高办电效率的工作意见》（以下简称“工作意见”），主要包括：
1.简化报装资料方面：居民、非居民、高压客户申请资料
种类，分别由2种、3种、4种均减少为1种，实行“一证受理”。
2.优化业扩流程方面：对低压客户，合并现场勘查和装表
接电环节，具备直接装表条件的，勘查确定供电方案后当场装表接电；不具备直接装表条件的，现场勘查时答复供电方案，根据与客户约定时间或电网配套工程竣工当日装表接电。对普通高压客户，取消设计审查和中间检查，并简化竣工检验内容。扩大直接开放容量范围，对10千伏及以下项目容量直接开放，取消原315千伏安免审批容量的限制。
3.配套工程建设方面：低压、高压业扩电网配套工程分别
按照抢修领料和“项目包”模式实施，并下放ERP系统权限。
4.推动透明公开方面：实行电网资源、业务进程、收费标
准信息的公开、透明，其中，电网资源信息通过公司网站、办公自动化平台、电子文件系统，实现内部共享；业务进程、收费标准信息通过营业厅、95598网站、手机APP、短信平台等渠道，实现对外公开。
5.强化内部考核方面：强化考核评价，国网客服中心负责
开展业扩客户回访；各级运监中心负责业扩报装专业协同质量和时限的监测；国网营销部、运监中心根据监测结果提出考核意见，并纳入业绩考核。
输出：
"""

resp = chat_comp.do(model="ERNIE-Speed-8K", messages=[{
    "role": "user",
    "content": prompt
}])

print(resp["body"]["result"])

```json
[{"问题":"公司当前面临的新形势是什么？","分析":"公司当前面临电力改革和分布式电源迅速发展的新形势，市场开拓和优质服务压力越来越大。", "答案":"公司当前面临电力改革和分布式电源迅速发展的新形势。", "依据":"当前，公司面临电力改革、分布式电源迅速发展的新形势，市场开拓和优质服务压力越来越大。"}, {"问题":"公司为什么要进一步解放思想、转变观念、创新机制？","分析":"公司需要解放思想、转变观念、创新机制以应对市场开拓和优质服务的压力，确保公司市场竞争优势。", "答案":"公司需要解放思想、转变观念、创新机制以应对市场开拓和优质服务的压力，确保公司市场竞争优势。", "依据":"当前，公司面临电力改革、分布式电源迅速发展的新形势，市场开拓和优质服务压力越来越大，迫切需要进一步解放思想，转变观念，创新机制。"}, {"问题":"《国家电网公司关于简化业扩手续提高办电效率深化为民服务的工作意见》的目的是什么？", "分析":"该意见旨在提升业扩报装效率、工作质量和服务水平，深入贯彻“你用电、我用心”的服务理念。", "答案":"该意见旨在提升业扩报装效率、工作质量和服务水平。", "依据":"为深入贯彻“你用电、我用心”服务理念，各单位认真落实《国家电网公司关于简化业扩手续提高办电效率深化为民服务的工作意见》。"}, {"问题":"《进一步精简业扩手续、提高办电效率的工作意见》主要包括哪些内容？", "分析":"该意见主要包括简化报装资料、优化业扩流程、配套工程建设、推动透明公开以及强化内部考核等方面的内容。", "答案":"该意见主要包括简化报装资料、优化业扩流程、配套工程建设措施等。", "依据":"《进一步精简业扩手续、提高办电效率的工作意见》主要包括：简化报装资料方面...配套工程建设方面。"}, {"问题":"在优化业扩流程方面，对于低压客户和高压客户分别采取了哪些措施？", "分析":"对于低压客户，合并了现场勘查和装表接电环节，具备直接装表条件的当场装表接电；对于高压客户，取消了设计审查和中间检查，简化了竣工检验内容。", "答案":"对于低压客户，合并了现场勘查和装表接电环节；对于高压客户，取消了设计审查和中间检查。", "依据":"优化业扩流程方面：对低压客户，合并现场勘查和装表接电环节...对普通高压客户，

In [15]:
prompt = f"""你是一个电力行业的专家，你的任务是从一段电力行业文本资料中抽取一些单选题，具体要求如下：
1、问题答案需要准确无误，其中正确的选项尽量使用原文内容
2、不要使用『其』『该文章』『该方法』等指代词，需要使用完整的名称
3、输出的每个问题需要包含：问题、A、B、C、D、分析、答案、答案的依据。
4、输出的格式为JSON格式，[{{"问题":"","A":"","B":"","C":"","D":"","分析":"","答案":"","依据":""}}]

##示例
输入：电力管理部门对危害供电、用电安全，扰乱正常供电、用电秩序的行为，除协助供电企业追缴电费外，应分别给予下列处罚：
1、擅自改变用电类别的，应责令其改正，给予警告；再次发生的，可下达中止供电命令，并处以一万元以下的罚款。
2、擅自超过合同约定的容量用电的，应责令其改正，给予警告；拒绝改正的，可下达中止供电命令，并按私增容量每千瓦（或每千伏安）100元，累计总额不超过五万元的罚款。
3、擅自超过计划分配的用电指标用电的，应责令其改正，给予警告，并按超用电力、电量分别处以每千瓦每次5元和每千瓦时10倍电度电价，累计总额不超过五万元的罚款；拒绝改正的，可下达中止供电命令。
4、擅自使用已经在供电企业办理暂停使用手续的电力设备，或者擅自启用已经被供电企业查封的电力设备的，应责令其改正，给予警告；启用电力设备危及电网安全的，可下达中止供电命令，并处以每次二万元以下的罚款。
输出：
```json
[{{"问题":"擅自改变用电类别的行为，应该受到什么处罚？","A":"罚款一万元","B":"首次发生的应责令其改正，给予警告；再次发生的，可下达中止供电命令，并处以一万元以下的罚款","C":"责令其改正，给予警告","D":"不予处罚","分析":"原文中提到『1、擅自改变用电类别的，应责令其改正，给予警告；再次发生的，可下达中止供电命令，并处以一万元以下的罚款。』和B选项一致，因此选择B","答案":"B","依据":"1、擅自改变用电类别的，应责令其改正，给予警告；再次发生的，可下达中止供电命令，并处以一万元以下的罚款。"}},{{"问题":"擅自超过合同约定的容量用电的行为，应该受到什么处罚？","A":"应责令其改正，给予警告；拒绝改正的，可下达中止供电命令，并按私增容量每千瓦（或每千伏安）100元，累计总额不超过五万元的罚款","B":"责令其改正给予警告","C":"不予处罚","D":"罚款十万元","分析":"原文中提到『2、擅自超过合同约定的容量用电的，应责令其改正，给予警告；拒绝改正的，可下达中止供电命令，并按私增容量每千瓦（或每千伏安）100元，累计总额不超过五万元的罚款。』，和A选项一致，因此选择A选项","答案":"A","依据":"2、擅自超过合同约定的容量用电的，应责令其改正，给予警告；拒绝改正的，可下达中止供电命令，并按私增容量每千瓦（或每千伏安）100元，累计总额不超过五万元的罚款。"}}]
```
输入：客户服务中心：
当前，公司面临电力改革、分布式电源迅速发展的新形势，
市场开拓和优质服务压力越来越大，迫切需要进一步解放思想，转变观念，创新机制，以客户需求为导向，精简办电程序，确保公司市场竞争优势。2014年，各单位认真落实《国家电网公司关于简化业扩手续提高办电效率深化为民服务的工作意见》（国家电网营销〔2014〕1049号）,提升了业扩报装效率、工作质量和服务水平。为深入贯彻“你用电、我用心”服务理念，最大限度地实现供电服务便民、为民、利民，提升优质服务水平，加快办电速度，公司研究制订了《进一步精简业扩手续、提高办电效率的工作意见》（以下简称“工作意见”），主要包括：
1.简化报装资料方面：居民、非居民、高压客户申请资料
种类，分别由2种、3种、4种均减少为1种，实行“一证受理”。
2.优化业扩流程方面：对低压客户，合并现场勘查和装表
接电环节，具备直接装表条件的，勘查确定供电方案后当场装表接电；不具备直接装表条件的，现场勘查时答复供电方案，根据与客户约定时间或电网配套工程竣工当日装表接电。对普通高压客户，取消设计审查和中间检查，并简化竣工检验内容。扩大直接开放容量范围，对10千伏及以下项目容量直接开放，取消原315千伏安免审批容量的限制。
3.配套工程建设方面：低压、高压业扩电网配套工程分别
按照抢修领料和“项目包”模式实施，并下放ERP系统权限。
4.推动透明公开方面：实行电网资源、业务进程、收费标
准信息的公开、透明，其中，电网资源信息通过公司网站、办公自动化平台、电子文件系统，实现内部共享；业务进程、收费标准信息通过营业厅、95598网站、手机APP、短信平台等渠道，实现对外公开。
5.强化内部考核方面：强化考核评价，国网客服中心负责
开展业扩客户回访；各级运监中心负责业扩报装专业协同质量和时限的监测；国网营销部、运监中心根据监测结果    提出考核意见，并纳入业绩考核。
输出：
"""

resp = chat_comp.do(model="ERNIE-Speed-8K", messages=[{
    "role": "user",
    "content": prompt
}])

print(resp["body"]["result"])

```json
[{"问题":"当前公司面临什么新形势？", "A":"电力改革、分布式电源迅速发展", "B":"市场开拓和优质服务压力增大", "C":"创新机制的需求迫切", "D":"以上都是", "分析":"文中提到“当前，公司面临电力改革、分布式电源迅速发展的新形势”，因此A和C都是正确的，同时这也导致了市场开拓和优质服务压力越来越大，所以正确答案是D选项。", "答案":"D", "依据":"当前，公司面临电力改革、分布式电源迅速发展的新形势"}
,{"问题":"关于简化业扩手续提高办电效率的工作意见是哪个公司在哪一年提出的？", "A":"国家电网公司、2013年", "B":"国家电网公司、2014年", "C":"地方电力公司、2014年", "D":"地方电力公司、2013年", "分析":"根据提供的信息，文中明确提到《国家电网公司关于简化业扩手续提高办电效率深化为民服务的工作意见》是在2014年提出的，所以正确答案是B选项。", "答案":"B", "依据":"《国家电网公司关于简化业扩手续提高办电效率深化为民服务的工作意见》（国家电网营销〔2014〕1049号）"}
,{"问题":"关于业扩报装流程的优化措施中，对于低压客户是怎么样调整的？", "A":"取消了现场勘查和装表接电环节", "B":"合并了现场勘查和装表接电环节，并当场装表接电", "C":"增加了现场勘查和装表接电的时间要求", "D":"简化了设计审查和中间检查流程", "分析":"在优化业扩报装流程的措施中，对于低压客户是“合并现场勘查和装表接电环节，具备直接装表条件的，勘查确定供电方案后当场装表接电”。因此，正确答案是B。", "答案":"B", "依据":"优化业扩流程方面：对低压客户，合并现场勘查和装表接电环节"}
,{"问题":"关于电网资源信息的公开途径有哪些？", "A":"公司网站、办公自动化平台、电子文件系统", "B":"内部会议、纸质文件、电子邮件系统", "C":"营业厅、社交媒体平台、电话热线", "D":"短信平台、手机APP、电视广播", "分析":"文中提到电网资源信息通过公司网站、办公自动化平台、电子文件系统实现内部共享。因此，正确答案是A。", "答案":"A", "依据":"推动透明公开方面：实行电网资源、业务进程、收费标準信息的公

In [16]:
check_prompt = f"""你是一个电力行业的专家，你的任务是根据原文信息来校验单项选择题的答案是否正确，具体要求如下：
1、你需要根据原文对问题进行思考分析，谨慎的判断答案结果是否正确
2、你判断的是单项选择题，如果输出多个选项，则判断为『错误』
3、输出的结果为json格式，{{"分析":"","结果:""}}
4、答案正确时结果字段为『正确』，答案错误时结果字段为『错误』

## 示例：
输入：
### 原文
客户服务中心：
当前，公司面临电力改革、分布式电源迅速发展的新形势，
市场开拓和优质服务压力越来越大，迫切需要进一步解放思想，转变观念，创新机制，以客户需求为导向，精简办电程序，确保公司市场竞争优势。2014年，各单位认真落实《国家电网公司关于简化业扩手续提高办电效率深化为民服务的工作意见》（国家电网营销〔2014〕1049号）,提升了业扩报装效率、工作质量和服务水平。为深入贯彻“你用电、我用心”服务理念，最大限度地实现供电服务便民、为民、利民，提升优质服务水平，加快办电速度，公司研究制订了《进一步精简业扩手续、提高办电效率的工作意见》（以下简称“工作意见”），主要包括：
1.简化报装资料方面：居民、非居民、高压客户申请资料
种类，分别由2种、3种、4种均减少为1种，实行“一证受理”。
2.优化业扩流程方面：对低压客户，合并现场勘查和装表
接电环节，具备直接装表条件的，勘查确定供电方案后当场装表接电；不具备直接装表条件的，现场勘查时答复供电方案，根据与客户约定时间或电网配套工程竣工当日装表接电。对普通高压客户，取消设计审查和中间检查，并简化竣工检验内容。扩大直接开放容量范围，对10千伏及以下项目容量直接开放，取消原315千伏安免审批容量的限制。
3.配套工程建设方面：低压、高压业扩电网配套工程分别
按照抢修领料和“项目包”模式实施，并下放ERP系统权限。
4.推动透明公开方面：实行电网资源、业务进程、收费标
准信息的公开、透明，其中，电网资源信息通过公司网站、办公自动化平台、电子文件系统，实现内部共享；业务进程、收费标准信息通过营业厅、95598网站、手机APP、短信平台等渠道，实现对外公开。
5.强化内部考核方面：强化考核评价，国网客服中心负责
开展业扩客户回访；各级运监中心负责业扩报装专业协同质量和时限的监测；国网营销部、运监中心根据监测结果提出考核意见，并纳入业绩考核。
### 题目
关于简化业扩手续提高办电效率的工作意见是哪个公司在哪一年提出的？
A：国家电网公司、2013年
B：国家电网公司、2014年
C：地方电力公司、2014年
D：地方电力公司、2013年
### 答案
B
输出：
```json
{{"分析":"根据提供的信息，文中明确提到《国家电网公司关于简化业扩手续提高办电效率深化为民服务的工作意见》是在2014年提出的，所以正确答案是B选项。","结果:"正确"}}
```
输入：
### 原文
客户服务中心：
当前，公司面临电力改革、分布式电源迅速发展的新形势，
市场开拓和优质服务压力越来越大，迫切需要进一步解放思想，转变观念，创新机制，以客户需求为导向，精简办电程序，确保公司市场竞争优势。2014年，各单位认真落实《国家电网公司关于简化业扩手续提高办电效率深化为民服务的工作意见》（国家电网营销〔2014〕1049号）,提升了业扩报装效率、工作质量和服务水平。为深入贯彻“你用电、我用心”服务理念，最大限度地实现供电服务便民、为民、利民，提升优质服务水平，加快办电速度，公司研究制订了《进一步精简业扩手续、提高办电效率的工作意见》（以下简称“工作意见”），主要包括：
1.简化报装资料方面：居民、非居民、高压客户申请资料
种类，分别由2种、3种、4种均减少为1种，实行“一证受理”。
2.优化业扩流程方面：对低压客户，合并现场勘查和装表
接电环节，具备直接装表条件的，勘查确定供电方案后当场装表接电；不具备直接装表条件的，现场勘查时答复供电方案，根据与客户约定时间或电网配套工程竣工当日装表接电。对普通高压客户，取消设计审查和中间检查，并简化竣工检验内容。扩大直接开放容量范围，对10千伏及以下项目容量直接开放，取消原315千伏安免审批容量的限制。
3.配套工程建设方面：低压、高压业扩电网配套工程分别
按照抢修领料和“项目包”模式实施，并下放ERP系统权限。
4.推动透明公开方面：实行电网资源、业务进程、收费标
准信息的公开、透明，其中，电网资源信息通过公司网站、办公自动化平台、电子文件系统，实现内部共享；业务进程、收费标准信息通过营业厅、95598网站、手机APP、短信平台等渠道，实现对外公开。
5.强化内部考核方面：强化考核评价，国网客服中心负责
开展业扩客户回访；各级运监中心负责业扩报装专业协同质量和时限的监测；国网营销部、运监中心根据监测结果提出考核意见，并纳入业绩考核。
### 问题
当前公司面临什么新形势？
A：电力改革、分布式电源迅速发展
B：市场开拓和优质服务压力增大
C：创新机制的需求迫切
D：以上都是
### 答案
D
输出：
"""

resp = chat_comp.do(model="ERNIE-Speed-8K", messages=[{
    "role": "user",
    "content": check_prompt
}])

print(resp["body"]["result"])

```json
{"分析":"原文第一句话即明确指出，公司当前面临电力改革、分布式电源迅速发展的新形势，同时也提到市场开拓和优质服务压力越来越大，以及迫切需要进一步解放思想，转变观念，创新机制。因此，选项D“以上都是”正确，涵盖了原文中提到的所有新形势。","结果":"正确"}
```


In [22]:
# 利用正则表达式从结果中提取我们需要的字段。
import re

def extract_json(md_string):
    pattern = r'```json(.*?)```'
    matches = re.findall(pattern, md_string, re.DOTALL)
    if len(matches) == 0 or not matches:
        return ""
    else:
        return matches[0]
json_string = extract_json(resp["body"]["result"])

正确


In [28]:
def prompt_format(content, question_type):
    if question_type == "single_choice":
        return f"""你是一个电力行业的专家，你的任务是从一段电力行业文本资料中抽取一些单选题，具体要求如下：
1、问题答案需要准确无误，其中正确的选项尽量使用原文内容
2、不要使用『其』『该文章』『该方法』等指代词，需要使用完整的名称
3、输出的每个问题需要包含：问题、A、B、C、D、分析、答案、答案的依据。
4、输出的格式为JSON格式，[{{"问题":"","A":"","B":"","C":"","D":"","分析":"","答案":"","依据":""}}]

##示例
输入：电力管理部门对危害供电、用电安全，扰乱正常供电、用电秩序的行为，除协助供电企业追缴电费外，应分别给予下列处罚：
1、擅自改变用电类别的，应责令其改正，给予警告；再次发生的，可下达中止供电命令，并处以一万元以下的罚款。
2、擅自超过合同约定的容量用电的，应责令其改正，给予警告；拒绝改正的，可下达中止供电命令，并按私增容量每千瓦（或每千伏安）100元，累计总额不超过五万元的罚款。
3、擅自超过计划分配的用电指标用电的，应责令其改正，给予警告，并按超用电力、电量分别处以每千瓦每次5元和每千瓦时10倍电度电价，累计总额不超过五万元的罚款；拒绝改正的，可下达中止供电命令。
4、擅自使用已经在供电企业办理暂停使用手续的电力设备，或者擅自启用已经被供电企业查封的电力设备的，应责令其改正，给予警告；启用电力设备危及电网安全的，可下达中止供电命令，并处以每次二万元以下的罚款。
输出：
```json
[{{"问题":"擅自改变用电类别的行为，应该受到什么处罚？","A":"罚款一万元","B":"首次发生的应责令其改正，给予警告；再次发生的，可下达中止供电命令，并处以一万元以下的罚款","C":"责令其改正，给予警告","D":"不予处罚","分析":"原文中提到『1、擅自改变用电类别的，应责令其改正，给予警告；再次发生的，可下达中止供电命令，并处以一万元以下的罚款。』和B选项一致，因此选择B","答案":"B","依据":"1、擅自改变用电类别的，应责令其改正，给予警告；再次发生的，可下达中止供电命令，并处以一万元以下的罚款。"}},{{"问题":"擅自超过合同约定的容量用电的行为，应该受到什么处罚？","A":"应责令其改正，给予警告；拒绝改正的，可下达中止供电命令，并按私增容量每千瓦（或每千伏安）100元，累计总额不超过五万元的罚款","B":"责令其改正给予警告","C":"不予处罚","D":"罚款十万元","分析":"原文中提到『2、擅自超过合同约定的容量用电的，应责令其改正，给予警告；拒绝改正的，可下达中止供电命令，并按私增容量每千瓦（或每千伏安）100元，累计总额不超过五万元的罚款。』，和A选项一致，因此选择A选项","答案":"A","依据":"2、擅自超过合同约定的容量用电的，应责令其改正，给予警告；拒绝改正的，可下达中止供电命令，并按私增容量每千瓦（或每千伏安）100元，累计总额不超过五万元的罚款。"}}]
```
输入：{content}
输出：
"""
    else:
        return f"""你是一个电力行业的专家，你的任务是从一段电力行业文本资料中抽取一些问答题，具体要求如下：
1、问题答案需要准确无误，问题、答案要求尽量来源于原文内容，不要改写
2、不要提出对于文档、原文的总结性问题，而是聚焦于具体的知识点
3、输出的每个问题需要包含：问题、分析、答案、依据
4、输出格式为json格式，[{{"问题":"","分析":"","答案":"","依据":""}}]

##示例
输入：水能是一种能源，是清洁能源.是绿色能源，是指水体的动能、势能和压力能等能量资源。水能是一种可再生能源，水能主要用于水力发电。水力发电将水的势能和动能装换成电能。以水力发电的工厂称为水力发电厂，简称水电厂，又称水电站，水力发电的优点是成本低、可连续再生、无污染。缺点是分布受水文、气候、地貌等自然条件的限制大。容易被地形、气候等多方面的因素所影响，国家还在研究如何更好的利用水能。广义的水能资源电活可流水域、蓄夕水能、放旋越、海越能等能是意义的水能资可流的水能资,是能源一难源,我们的才能是钢动能人们是易开放用的域成重的能也是确流能源。
水落差在重力作用下形能,从河流或水车转等事立水源河立水利脉水的压力或者速材水物机之,从而常水增转状水机能燃后再水物将带动发电机旋转能磁力线产生交流转。而低处的水通过阳光照射，形成水蒸气，循环到地球各处从而恢复高位水源的水分布。水不仅可以直接被人类利用，它还是能量的载体，太阳能驱动地球上水循环使之持续进行，地表水的流动是重要的一环在落差大、流量大的地区，水能资源丰富。随着矿物燃料的日渐减少，水能是非常重要目前景广阔的替代资源，世界上水力发电还处于起步阶段。河流、潮汐、波浪以及涌浪等水运动均可以用来发电，也有部分水能用于灌溉。
输出：
```json
[{{"问题":"水能的主要应用场景是什么？","分析":"原文中提到“水能是一种可再生能源水能主要用于水力发电”，因此水能主要应用于水力发电。","答案":"水能主要用于水力发电","依据":"水能是一种可再生能源，水能主要用于水力发电。"}},{{"问题":"水力发电的优点是什么？","分析":"原文中提到“水力发电的优点是成本低、可连续再生、无污染。”，所以得出水力发电的优点是成本低、可连续再生、无污染。","答案":"水力发电的优点是成本低、可连续再生、无污染","依据":"水力发电的优点是成本低、可连续再生、无污染。"}}]
```

输入：空气是风冷散热系统最常用的冷却介质此外，还可以采用气和氮气对电机进行冷却，XU等采用氟气对高温气冷堆的鼓风机驱动电机进行冷却，电机额定功率为4500kW,仿真结果表明氮气的初始温度显著影响电机的温升，氮气和氢气的热导率是空气的6-7倍，因此相同的气体温度和流速情况下氮气、氩气冷却的散热效率可以达到控气的1.3-1.5倍，同时氢气和氟气的密度只有空气的1/14-177因此采用氢气、氮气代替空气可以显著减小冷却系统的通风损耗和风摩损耗，然而氢气和氮气高昂的价格、复杂的储存供应系统限制了氢气、氮气冷却系统的大规模应用，此外氢气冷却系统需要氢气纯度不小于96%和安全可靠的密封防爆系统，这进一步限制了氢气冷却系统的应用。因此，氢气、氨气冷却系统多应用于大功率发电机、高温反应堆等特殊场合。
输出：
```json
["问题":"电机散热系统中，采用氢气或氟气作为冷却介质的优势和限制分别是什么？","分析":"在原文中提到“氮气和氯气的热导率是空气的6-7倍，因此相同的气体温度和流速情况下氮气、氯气冷却的散热效率可以达到空气的1.3-1.5倍。同时氢气和氮气的密度只有空气的1/14-1/7因此采用氢气、氮气代替空气可以显著减小冷却系统的通风损耗和风摩损耗。”、“然而氢气和气高昂的价格、复杂的储存供应系统限制了氩气、氩气冷却系统的大规模应用。此外氢气冷却系统需要氢气纯度不小于96%和安全可靠的密封防爆系统这进一步眼制了氢气冷却系统的应用”等描述，因此得出答案是“使用氢气或气作为冷却介质的优势是：氮气和氢气的热导率是空气的6-7倍因此相同的气体温度和流速情况下氟气、氮气冷却的散热效率可以达到空气的1.3-15倍。同时，氢气和氮气的密度只有空气的1/14-1/7因此采用氢气、氢气代替空气可以显著减小冷却系统的通风损耗和风摩损耗；限制是：氢气和氮气高昂的价格、复杂的储存供应系统限制了氢气、氮气冷系统的大规模应用、此外氢气冷却系统需要氢气纯度不小于96%和安全可靠的密封防爆系统这进一步限制了氮气冷却系统的应用","答案":"使用氢气或氟气作为冷却介质的优势是：气和氢气的热导率是空气的6-7倍因此相同的气体温度和流速情况下，氮气、氮气冷却的散热效率可以达到空气的1.3-1.5倍。同时氢>气和氮气的密度只有空气的1/14-1/7因此采用氮气、氢气代替空气可以显著减小冷却系统的通风损耗和风摩损耗；限制是：氢气和氮气高昂的价格、复杂的储存供应系统限制了氮气、气冷却系统的大规模应用。此外，氢气冷却系统需要氢气纯度不小于96%和安全可靠的密封防爆系统这进一步限制了氢气冷却系统的应用","依据":"气和氢气的热导率是空气的6-7倍，因此相同的气体温度和流速情况下，氟气、氩气冷却的散热效率可以达到空气的1.3-1.5倍，同时氢气和氟气的密度只有空气的1/14-1/7因此采用氢气、氮气代替空气可以显著减小冷却系统的通风损耗和风摩损耗，然而氩气和氮气高昂的价格、复杂的储存供应系统限制了氩气、氟气冷却系统的大规模应用，此外氢气冷却系统需要氢气纯度不小于96%和安全可靠的密封防爆系统，这进一步限制了氢气冷却系统的应用"}}]
```
输入：{content}
输出：
"""

def recheck_prompt_format(question_data, question_type, content):
    if question_type == "single_choice":
        return f"""你是一个电力行业的专家，你的任务是根据原文信息来校验单项选择题的答案是否正确，具体要求如下：
1、你需要根据原文对问题进行思考分析，谨慎的判断答案结果是否正确
2、你判断的是单项选择题，如果输出多个选项，则判断为『错误』
3、输出的结果为json格式，{{"分析":"","结果:""}}
4、答案正确时结果字段为『正确』，答案错误时结果字段为『错误』

## 示例：
输入：
### 原文
客户服务中心：
当前，公司面临电力改革、分布式电源迅速发展的新形势，
市场开拓和优质服务压力越来越大，迫切需要进一步解放思想，转变观念，创新机制，以客户需求为导向，精简办电程序，确保公司市场竞争优势。2014年，各单位认真落实《国家电网公司关于简化业扩手续提高办电效率深化为民服务的工作意见》（国家电网营销〔2014〕1049号）,提升了业扩报装效率、工作质量和服务水平。为深入贯彻“你用电、我用心”服务理念，最大限度地实现供电服务便民、为民、利民，提升优质服务水平，加快办电速度，公司研究制订了《进一步精简业扩手续、提高办电效率的工作意见》（以下简称“工作意见”），主要包括：
1.简化报装资料方面：居民、非居民、高压客户申请资料
种类，分别由2种、3种、4种均减少为1种，实行“一证受理”。
2.优化业扩流程方面：对低压客户，合并现场勘查和装表
接电环节，具备直接装表条件的，勘查确定供电方案后当场装表接电；不具备直接装表条件的，现场勘查时答复供电方案，根据与客户约定时间或电网配套工程竣工当日装表接电。对普通高压客户，取消设计审查和中间检查，并简化竣工检验内容。扩大直接开放容量范围，对10千伏及以下项目容量直接开放，取消原315千伏安免审批容量的限制。
3.配套工程建设方面：低压、高压业扩电网配套工程分别
按照抢修领料和“项目包”模式实施，并下放ERP系统权限。
4.推动透明公开方面：实行电网资源、业务进程、收费标
准信息的公开、透明，其中，电网资源信息通过公司网站、办公自动化平台、电子文件系统，实现内部共享；业务进程、收费标准信息通过营业厅、95598网站、手机APP、短信平台等渠道，实现对外公开。
5.强化内部考核方面：强化考核评价，国网客服中心负责
开展业扩客户回访；各级运监中心负责业扩报装专业协同质量和时限的监测；国网营销部、运监中心根据监测结果提出考核意见，并纳入业绩考核。
### 题目
关于简化业扩手续提高办电效率的工作意见是哪个公司在哪一年提出的？
A：国家电网公司、2013年
B：国家电网公司、2014年
C：地方电力公司、2014年
D：地方电力公司、2013年
### 答案
B
输出：
```json
{{"分析":"根据提供的信息，文中明确提到《国家电网公司关于简化业扩手续提高办电效率深化为民服务的工作意见》是在2014年提出的，所以正确答案是B选项。","结果:"正确"}}
```
输入：
### 原文
{content}
### 问题
{question_data["问题"]}
A：{question_data["A"]}
B：{question_data["B"]}
C：{question_data["C"]}
D：{question_data["D"]}
### 答案
{question_data["答案"]}
输出：
"""

def split_string(s, length):
    return [s[i:i+length] for i in range(0, min(len(s), 3*length), length)]

with open("/Users/hanjie05/Documents/项目文档/无锡培训/国家电网公司关于印发《进一步精简业扩手续、提高办电效率的工作意见》的通知(国家电网营销〔2015〕70号)(1)(1)(1).md", "r") as f:
    doc_content  = f.read()

contents = split_string(doc_content, 1000)
cnt = 0
for content in contents:
    cnt += 1
    print("Processing:" + str(cnt))
    # 生成抽取的问题
    prompt = prompt_format(content, "single_choice")
    resp = chat_comp.do(model="ERNIE-Speed-8K", messages=[{
        "role": "user",
        "content": prompt
    }])
    json_string = extract_json(resp["body"]["result"])
    try:
        results = json.loads(json_string)
    except Exception as e:
        print(json_string)
        print(e)
        continue

    for result in results:
        check_prompt = recheck_prompt_format(result, "single_choice", content)
        resp = chat_comp.do(model="ERNIE-Speed-8K", messages=[{
            "role": "user",
            "content": check_prompt
        }])
        json_string_2 = extract_json(resp["body"]["result"])
        try:
            check_result = json.loads(json_string_2)
        except Exception as e:
            print(json_string)
            print(e)
            continue
        if check_result["结果"] == "错误":
            continue
        with open("./result.jsonl", "a") as f_out:
            f_out.write(json.dumps(result, ensure_ascii=False) + "\n")


Processing:1

[{"问题":"国家电网公司关于精简业扩手续提高办电效率的工作意见是什么时候印发的？","A":"2014年","B":"2015年","C":"根据公司需求随时印发","D":"2016年","分析":"根据提供的资料，国家电网公司关于精简业扩手续提高办电效率的工作意见是在2015年印发的。","答案":"B","依据":"国家电网营销〔2015〕70号文件标题明确提到是2015年的工作意见。"}, {"问题":"关于进一步精简业扩手续的工作意见中，对于低压客户的业扩流程做了哪些优化？", "A":"合并现场勘查和装表接电环节，具备直接装表条件的当场装表接电","B":"取消了设计审查和中间检查，简化了竣工检验内容","C":"扩大了直接开放容量范围，取消了原315千伏安免审批容量的限制","D":"以上都是","分析":"根据提供的文件内容，关于进一步精简业扩手续的工作意见中，确实对低压客户的业扩流程做了以上所有的优化。","答案":"D","依据":"文件提到的优化业扩流程方面，明确提到了对低压客户的这些优化措施。"}, {"问题":"关于电网资源信息的公开渠道有哪些？", "A":"公司网站、办公自动化平台、电子文件系统","B":"只通过公司网站对外公开","C":"通过内部会议告知","D":"通过手机短信平台公开","分析":"根据提供的文件内容，电网资源信息通过公司网站、办公自动化平台、电子文件系统，实现内部共享。因此，选项A更全面地涵盖了公开渠道。", "答案":"A", "依据":"文件推动透明公开方面明确提到了电网资源信息的公开渠道。"}, {"问题":"关于精简业扩手续提高办电效率的工作意见中的‘一证受理’指的是什么？", "A":"只需要提供一份证明文件即可办理业务","B":"必须使用电子证照进行办理","C":"需要多次现场核实资料真实性","D":"可以免去所有的审批手续","分析":"根据提供的文件内容，‘一证受理’指的是居民、非居民、高压客户申请资料种类分别由多种减少为一种，实行一种证明文件即可受理。因此，选项A正确。", "答案":"A", "依据":"文件简化报装资料方面明确提到了实行‘一证受理’。"}]

Expecting ':' delimiter: line 2 column 112 (ch

In [29]:
!pip install datasketch

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [30]:
from datasketch import MinHash, MinHashLSH
import json

def is_duplicate_with_lsh(data, threshold, num_perm=128):
    """
    判断数据是否有重复
    :param data: 数据
    :param threshold: 阈值
    :param num_perm: MinHash的参数
    :return: 重复数据的索引
    """
    lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)
    minhashes = {}

    for c, i in enumerate(data):
        minhash = MinHash(num_perm=num_perm)
        for d in i:
            minhash.update(d.encode('utf8'))
        lsh.insert(c, minhash)
        minhashes[c] = minhash

    duplicate = []
    for i in range(len(data)):
        result = lsh.query(minhashes[i])
        if len(result) > 1:
            duplicate.append(result)

    return duplicate

data = ["你好", "你好吗", "今天天气不错"]
print(is_duplicate_with_lsh(data, 0.5))

[[0, 1], [0, 1]]


In [ ]:
from datasketch import MinHash, MinHashLSH
import json

def is_duplicate_with_lsh(data, threshold, num_perm=128):
    """
    判断数据是否有重复
    :param data: 数据
    :param threshold: 阈值
    :param num_perm: MinHash的参数
    :return: 重复数据的索引
    """
    lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)
    minhashes = {}

    for c, i in enumerate(data):
        minhash = MinHash(num_perm=num_perm)
        for d in i:
            minhash.update(d.encode('utf8'))
        lsh.insert(c, minhash)
        minhashes[c] = minhash

    duplicate = []
    for i in range(len(data)):
        result = lsh.query(minhashes[i])
        if len(result) > 1:
            duplicate.append(result)

    return duplicate


datas = []
data_map = {}
with open("/Users/hanjie05/Documents/项目文档/国网大模型/exam_1128_fill_no_image_analysis.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)
        datas.append(data[0]["prompt"])


# 测试
dups_indexs = is_duplicate_with_lsh(datas, 0.7)
for i in dups_indexs[10]:
    print(datas[i])
cnt_map = {}
for sub_index in dups_indexs:
    for index in sub_index:
        if index not in cnt_map:
            cnt_map[index] = 1
        else:
            cnt_map[index] += 1

deleted_index = []
for sub_index in dups_indexs:
    max_value = max([cnt_map[i] for i in sub_index])
    deleted_index.extend([i for i in sub_index if cnt_map[i] != max_value])
# 对 deleted_index 进行去重
deleted_index = list(set(deleted_index))
deleted_query = [datas[i] for i in deleted_index]
with open("/Users/hanjie05/Documents/项目文档/国网大模型/exam_1128_fill_no_image_analysis.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)
        if data[0]["prompt"] in deleted_query:
            continue
        with open("/Users/hanjie05/Documents/项目文档/国网大模型/exam_1128_fill_no_image_analysis_dup.jsonl", "a") as wf:
            wf.write(line)
# [[0,1,2],[1,2,3],[3,2,1]]
# 统计每个index出现的次数，每个sublist中，保留出现次数最多的元素
